<a href="https://colab.research.google.com/github/Aynahd/VeriAssist/blob/main/emotionsTry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('emotions.csv')
print(df)

             id                                               text  \
0       eew5j0j                                    That game hurt.   
1       eemcysk   >sexuality shouldn’t be a grouping category I...   
2       ed2mah1     You do right, if you don't care then fuck 'em!   
3       eeibobj                                 Man I love reddit.   
4       eda6yn6  [NAME] was nowhere near them, he was by the Fa...   
...         ...                                                ...   
140747  edi4nbj  Because Rian is a hack...and TLJ sucks. That's...   
140748  efbxvgo       The tree actually moved!!! Who is this guy?!   
140749  edodrci  Wait his nostrils are plugged? Isn’t that the ...   
140750  ed387lu  My great aunt was saved by the serum delivered...   
140751  edkr8ou  Nothing wrong with striving. Forcibly making p...   

       example_very_unclear  admiration  amusement  anger  annoyance  \
0                     False         0.0        0.0    0.0        0.0   
1              

<ipython-input-4-8772a0132bf0>:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('emotions.csv')


In [17]:
import re
import torch
from torch.utils.data import Dataset
from nltk.stem import PorterStemmer
from nltk import pos_tag, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk import pos_tag, word_tokenize
from nltk.corpus import stopwords
import spacy
nlp = spacy.load("en_core_web_sm")

import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')
import re
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.tag import pos_tag
import spacy

nlp = spacy.load("en_core_web_sm")

def process(text_series):

    text_series = text_series.str.lower()

    def process(text):
        text = re.sub(r"[^a-zA-Z\s]", "", text)
        tokens = word_tokenize(text)

        stop_words = {"in", "on", "the", "a", "an", "at", "with", "my"}
        tokens = [token for token in tokens if token not in stop_words]

        stemmer = PorterStemmer()
        tokens = [stemmer.stem(token) for token in tokens]

        tagged = pos_tag(tokens)
        filtered = [word for word, tag in tagged if tag.startswith(('NN', 'JJ', 'VB'))]

        tokens = [stemmer.stem(word) for word in filtered]
        return tokens

    return text_series.apply(process)



/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [6]:
text = df['text'].iloc[:5]
print(text)

0                                      That game hurt.
1     >sexuality shouldn’t be a grouping category I...
2       You do right, if you don't care then fuck 'em!
3                                   Man I love reddit.
4    [NAME] was nowhere near them, he was by the Fa...
Name: text, dtype: object


In [7]:
tokens = process(text)
print(tokens)

0                                         [game, hurt]
1    [sexual, shouldnt, be, group, categori, make, ...
2                                 [do, dont, care, em]
3                               [man, i, love, reddit]
4                               [name, wa, wa, falcon]
Name: text, dtype: object


In [8]:
import spacy
nlp = spacy.load("en_core_web_sm")

def ner_anonymizer(text):
    doc = nlp(text)

    anonymized_text = text
    for ent in doc.ents:
        if ent.label_ in {"PERSON", "GPE", "DATE", "ORG"}:
            anonymized_text = anonymized_text.replace(ent.text, f"<{ent.label_}>")

    return anonymized_text


In [9]:
import pandas as pd

emotion_columns = [
    'admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring',
    'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval',
    'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief',
    'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief',
    'remorse', 'sadness', 'surprise', 'neutral'
]
df['emotion'] = df[emotion_columns].idxmax(axis=1)

df2 = df.drop(columns=emotion_columns)

print(df2)


             id                                               text  \
0       eew5j0j                                    That game hurt.   
1       eemcysk   >sexuality shouldn’t be a grouping category I...   
2       ed2mah1     You do right, if you don't care then fuck 'em!   
3       eeibobj                                 Man I love reddit.   
4       eda6yn6  [NAME] was nowhere near them, he was by the Fa...   
...         ...                                                ...   
140747  edi4nbj  Because Rian is a hack...and TLJ sucks. That's...   
140748  efbxvgo       The tree actually moved!!! Who is this guy?!   
140749  edodrci  Wait his nostrils are plugged? Isn’t that the ...   
140750  ed387lu  My great aunt was saved by the serum delivered...   
140751  edkr8ou  Nothing wrong with striving. Forcibly making p...   

       example_very_unclear     emotion  
0                     False     sadness  
1                      True  admiration  
2                     False     n

<ipython-input-9-32fd403a09bf>:10: FutureWarning: The behavior of DataFrame.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  df['emotion'] = df[emotion_columns].idxmax(axis=1)


In [10]:
df3 = df2[df2['example_very_unclear'] == False].drop(columns=['example_very_unclear'])
print(df3)

             id                                               text     emotion
0       eew5j0j                                    That game hurt.     sadness
2       ed2mah1     You do right, if you don't care then fuck 'em!     neutral
3       eeibobj                                 Man I love reddit.        love
4       eda6yn6  [NAME] was nowhere near them, he was by the Fa...     neutral
5       eespn2i  Right? Considering it’s such an important docu...   gratitude
...         ...                                                ...         ...
140746  eev3dba  And yet the guy proclaiming he understands it ...  admiration
140747  edi4nbj  Because Rian is a hack...and TLJ sucks. That's...   amusement
140748  efbxvgo       The tree actually moved!!! Who is this guy?!    surprise
140749  edodrci  Wait his nostrils are plugged? Isn’t that the ...     neutral
140750  ed387lu  My great aunt was saved by the serum delivered...     neutral

[138516 rows x 3 columns]


In [11]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df3['emotion_encoded'] = label_encoder.fit_transform(df3['emotion'])
print(df3)


             id                                               text  \
0       eew5j0j                                    That game hurt.   
2       ed2mah1     You do right, if you don't care then fuck 'em!   
3       eeibobj                                 Man I love reddit.   
4       eda6yn6  [NAME] was nowhere near them, he was by the Fa...   
5       eespn2i  Right? Considering it’s such an important docu...   
...         ...                                                ...   
140746  eev3dba  And yet the guy proclaiming he understands it ...   
140747  edi4nbj  Because Rian is a hack...and TLJ sucks. That's...   
140748  efbxvgo       The tree actually moved!!! Who is this guy?!   
140749  edodrci  Wait his nostrils are plugged? Isn’t that the ...   
140750  ed387lu  My great aunt was saved by the serum delivered...   

           emotion  emotion_encoded  
0          sadness               26  
2          neutral               20  
3             love               18  
4      

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

In [19]:
from nltk.tokenize import word_tokenize
df3['tokens'] = df3['text'].apply(word_tokenize)


In [21]:
from gensim.models import Word2Vec

word2vec_model = Word2Vec(sentences=df3['tokens'], vector_size=100, window=5, min_count=1, workers=4)
word2vec_model.save("word2vec.model")


In [22]:
import numpy as np


def sentence_to_vector(tokens, model):
    vectors = [model.wv[word] for word in tokens if word in model.wv]
    if len(vectors) == 0:
        return np.zeros(model.vector_size)
    return np.mean(vectors, axis=0)

df3['sentence_vector'] = df3['tokens'].apply(lambda tokens: sentence_to_vector(tokens, word2vec_model))

X = np.vstack(df3['sentence_vector'].values)
y = df3['emotion_encoded']


In [23]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

classifier = RandomForestClassifier(n_estimators=100, random_state=42)
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.33973433439214556
Classification Report:
               precision    recall  f1-score   support

           0       0.45      0.44      0.44      2301
           1       0.45      0.40      0.42      1217
           2       0.30      0.26      0.28      1053
           3       0.18      0.16      0.17      1541
           4       0.21      0.17      0.19      2065
           5       0.23      0.18      0.20       719
           6       0.26      0.21      0.24       900
           7       0.34      0.29      0.31      1029
           8       0.19      0.18      0.18       376
           9       0.15      0.12      0.14       923
          10       0.22      0.18      0.20      1143
          11       0.18      0.13      0.15       445
          12       0.18      0.13      0.15       232
          13       0.25      0.19      0.21       621
          14       0.30      0.26      0.28       327
          15       0.56      0.56      0.56      1085
          16       0.14    